## Example of how to work with the location data in the timeline dump data

This has been tested with the following configuration:

```
- branca=0.4.1
- folium=0.11.0
- ipython=7.17.0
- jupyter=1.0.0
- pandas=1.1.0
```

An easy approach to set all the dependencies is:
- use the `setup/environment36.yml` and `setup/environment36.notebook.additions.yml` in https://github.com/e-mission/e-mission-server/, OR
- set the `EMISSION_SERVER_HOME` environment variable and then run:
   - `bash setup.sh`
   - `source activate.sh`
   - `./em-jupyter-notebook.sh`

In [ ]:
import folium
import json
import pandas as pd
import numpy as np

In [ ]:
print("folium %s" % folium.__version__)
print("json %s" % json.__version__)
print("pandas %s" % pd.__version__)
print("numpy %s" % np.__version__)

### Get the file from your phone

Get a timeline dump by using the "Download json dump" option from the profile

In [ ]:
TEST_FILE = "/tmp/2023-06-05.2023-06-05.timeline.iphone"
all_entries = json.load(open(TEST_FILE))

### Get raw and filtered locations

Filtered locations are filtered based on the accuracy of the point

In [ ]:
raw_loc_df = pd.json_normalize([e for e in all_entries if e["metadata"]["key"] == "background/location"])

In [ ]:
raw_loc_df.columns

In [ ]:
raw_loc_df.head()

In [ ]:
filtered_loc_df = pd.json_normalize([e for e in all_entries if e["metadata"]["key"] == "background/filtered_location"])

In [ ]:
filtered_loc_df.columns

In [ ]:
filtered_loc_df.head()

In [ ]:
len(raw_loc_df), len(filtered_loc_df)

In [ ]:
raw_loc_df['data.loc.coordinates']

In [ ]:
geojson_linestring = {"type": "LineString", "coordinates": raw_loc_df['data.loc.coordinates'].to_list()}; geojson_linestring

### Visualize them using folium


In [ ]:
m = folium.Map()
m.add_child(folium.GeoJson(geojson_linestring))

### Look at the time difference

In [ ]:
ts_diff = raw_loc_df["data.ts"].diff()

In [ ]:
ts_diff.hist()

### Narrow down to the location with the max density

This timeline was generated on a phone with duty cycling turned on, so we did not capture any data while not in motion. So there are a few points with a large diff. Let's filter until we get to the the highest density occurs.

In [ ]:
ts_diff[ts_diff < 2000].hist()

In [ ]:
ts_diff[ts_diff < 50].hist()

In [ ]:
ts_diff[ts_diff < 10].hist()

In [ ]:
ts_diff[ts_diff < 2].hist()

In [ ]:
ts_diff[np.logical_and(ts_diff < 1.25, ts_diff > 0.75)].hist()